In [1]:
import pandas as pd

In [2]:
data_df = pd.read_csv("top2.csv",index_col=0)
data_df = data_df[data_df["change"] < 1.22]
data_df = data_df[~data_df["minute_low"].isnull()]
data_df = data_df[~data_df.index.duplicated()] 
data_df = data_df[data_df["st"] == False]
data_df = data_df[data_df["isnew"] == False]
data_df = data_df[data_df["low2"] < data_df["high_limit2"]]
# data_df = data_df[data_df["open3"] < data_df["close2"] * 1.05]
# data_df = data_df[data_df["open3"] > data_df["close2"] * 0.95]
df = data_df

In [3]:
df.head(3)

,change,date,stock,close0,high0,high_limit0,low0,open0,volume0,day0,...,volume3,day3,minute_low,minute_volume,minute,recent,circap,isnew,capq,st
000001.XSHE_2008-09-22,0.957116,2008-09-22 00:00:00,000001.XSHE,13.30,13.98,14.38,12.40,12.51,29019241.0,390.0,...,20571693.0,393.0,4.22,0.0,09:30:00,2.0,1579.0,0.0,0.478775,False
000002.XSHE_2007-05-09,1.017367,2007-05-09 00:00:00,000002.XSHE,18.08,18.25,19.75,17.80,17.88,60348532.0,76.0,...,80033729.0,79.0,6.90,400574072.0,14:16:00,2.0,1446.0,0.0,0.438448,False
000002.XSHE_2008-04-25,0.957360,2008-04-25 00:00:00,000002.XSHE,20.15,20.38,21.23,18.35,18.79,76694971.0,309.0,...,89853741.0,312.0,11.81,273057847.0,13:16:00,2.0,1549.0,0.0,0.469679,False


In [4]:
df["change3"] = df["close3"]/df["close2"]
df["hchange3"] = df["high3"]/df["close2"]
df["inhigh3"] = df["high3"]/df["open3"]
df["inlow3"] = df["low3"]/df["open3"]
df["volume_r"] = df["volume2"]/df["volume1"]
df["open_change"] = df["open3"]/df["close2"]

# 开盘价对比

In [19]:
keys = ["high","not_high","low","not_low","top","bottom","up","down","low_top","high_bottom","close"]
def evaluate_df(sub_df):
    high_df = sub_df[sub_df["inhigh3"] >= 1.02]
    not_high_df = sub_df[sub_df["inhigh3"] < 1.02]
    low_df = sub_df[sub_df["inlow3"] <= 0.98]
    not_low_df = sub_df[sub_df["inlow3"] > 0.98]
    top_df = sub_df[sub_df["change3"] > 1.095]
    bottom_df = sub_df[sub_df["change3"] < 0.905]
    down_df = sub_df[sub_df["inhigh3"] == 1.0]
    up_df = sub_df[sub_df["inlow3"] == 1.0]
    low_top_df = sub_df[(sub_df["inlow3"] <= 0.98) & (sub_df["change3"] > 1.095)]
    high_bottom_df = sub_df[(sub_df["inhigh3"] >= 1.02) & (sub_df["change3"] < 0.905)]
    data_dic = {}
    index = 0
    for ssub_df in [high_df,not_high_df,low_df,not_low_df,top_df,bottom_df,up_df,down_df,low_top_df,high_bottom_df]:
        count = float(len(ssub_df))/len(sub_df)
        data_dic[keys[index]] = count
        index = index + 1
    data_dic["close"] = (sub_df["close3"]/sub_df["open3"]).mean()
    return pd.Series(data_dic)

In [20]:
result_df = pd.DataFrame(columns = keys)
for i in range(0,20):
    up_limit = 1.1 - float(i) * 0.01
    down_limit = 1.09 - float(i) * 0.01
    sub_df = df[(df["open_change"] > down_limit) & (df["open_change"] < up_limit)]
    sub_df1 = sub_df[sub_df["volume_r"] >= 1.0]
    sub_df2 = sub_df[sub_df["volume_r"] < 1.0]
    se1 = evaluate_df(sub_df1)
    se2 = evaluate_df(sub_df2) 
    result_df.ix[str(down_limit) + "-" + str(up_limit) + "放量"] = se1
    result_df.ix[str(down_limit) + "-" + str(up_limit) + "缩量"] = se2
result_df

,high,not_high,low,not_low,top,bottom,up,down,low_top,high_bottom,close
1.09-1.1放量,0.000000,1.000000,0.735395,0.264605,0.670103,0.013746,0.182131,0.577320,0.405498,0.000000,0.975308
1.09-1.1缩量,0.000000,1.000000,0.744186,0.255814,0.745736,0.009302,0.179845,0.610853,0.491473,0.000000,0.981352
1.08-1.09放量,0.000000,1.000000,0.907692,0.092308,0.492308,0.007692,0.007692,0.207692,0.400000,0.000000,0.971497
1.08-1.09缩量,0.000000,1.000000,0.918919,0.081081,0.625483,0.007722,0.007722,0.135135,0.544402,0.000000,0.985308
1.07-1.08放量,0.636364,0.363636,0.880682,0.119318,0.505682,0.017045,0.011364,0.187500,0.386364,0.000000,0.985641
1.07-1.08缩量,0.685619,0.314381,0.913043,0.086957,0.558528,0.020067,0.006689,0.100334,0.474916,0.010033,0.987183
1.06-1.07放量,0.778195,0.221805,0.812030,0.187970,0.466165,0.011278,0.011278,0.116541,0.285714,0.000000,0.993673
1.06-1.07缩量,0.833333,0.166667,0.841954,0.158046,0.563218,0.002874,0.022989,0.066092,0.410920,0.000000,0.998374
1.05-1.06放量,0.807882,0.192118,0.788177,0.211823,0.433498,0.009852,0.036946,0.056650,0.238916,0.000000,0.998545
1.05-1.06缩量,0.867971,0.132029,0.777506,0.222494,0.537897,0.004890,0.051345,0.051345,0.337408,0.002445,1.006805


In [10]:
print result_df

                 high  not_high       low   not_low       top    bottom
1.09-1.1放量   0.000000  1.000000  0.735395  0.264605  0.670103  0.013746
1.09-1.1缩量   0.000000  1.000000  0.744186  0.255814  0.745736  0.009302
1.08-1.09放量  0.000000  1.000000  0.907692  0.092308  0.492308  0.007692
1.08-1.09缩量  0.000000  1.000000  0.918919  0.081081  0.625483  0.007722
1.07-1.08放量  0.636364  0.363636  0.880682  0.119318  0.505682  0.017045
1.07-1.08缩量  0.685619  0.314381  0.913043  0.086957  0.558528  0.020067
1.06-1.07放量  0.778195  0.221805  0.812030  0.187970  0.466165  0.011278
1.06-1.07缩量  0.833333  0.166667  0.841954  0.158046  0.563218  0.002874
1.05-1.06放量  0.807882  0.192118  0.788177  0.211823  0.433498  0.009852
1.05-1.06缩量  0.867971  0.132029  0.777506  0.222494  0.537897  0.004890
1.04-1.05放量  0.830508  0.169492  0.764407  0.235593  0.388136  0.016949
1.04-1.05缩量  0.820734  0.179266  0.790497  0.209503  0.399568  0.021598
1.03-1.04放量  0.800484  0.199516  0.759371  0.240629  0.291415  0

In [5]:
sub_df = df[(df["high3"] == df["open3"]) & (df["close3"] < df["open3"])]
print sub_df["open_change"].mean()
print sub_df["open_change"].hist(bins=100)

1.01974115098
Axes(0.125,0.11;0.775x0.77)


In [5]:
df["open_change"].hist(bins=100)

In [6]:
df["open_change"].mean()

1.0068894288504737

# 第二天缩量的股票和放量的股票对比

In [5]:
print df[df["volume_r"] > 1.0]["change3"].mean()
print df[df["volume_r"] < 1.0]["change3"].mean()

1.00265320955
1.0193275187


In [6]:
print df[df["volume_r"] > 1.0]["hchange3"].mean()
print df[df["volume_r"] < 1.0]["hchange3"].mean()

1.04574648976
1.06657890862


In [7]:
print df[df["volume_r"] > 1.0]["inhigh3"].mean()
print df[df["volume_r"] < 1.0]["inhigh3"].mean()

1.05050368814
1.0526207906


In [8]:
print df[df["volume_r"] > 1.0]["inlow3"].mean()
print df[df["volume_r"] < 1.0]["inlow3"].mean()

0.960036265755
0.954263162112


In [9]:
def cal_ratio(se):
    valid_se = se[se < 0.98]
    ratio = float(len(valid_se))/len(se)
    return ratio

In [10]:
print cal_ratio(df[df["volume_r"] > 1.0]["inlow3"])
print cal_ratio(df[df["volume_r"] < 1.0]["inlow3"])

0.729286574756
0.755342902711


In [17]:
def cal_ratio(se):
    valid_se = se[se > 1.02]
    ratio = float(len(valid_se))/len(se)
    return ratio

In [20]:
print cal_ratio(df[df["volume_r"] > 3.0]["inhigh3"])
print cal_ratio(df[df["volume_r"] < 0.2]["inhigh3"])

0.725590955807
1.0


In [15]:
valid_df = df[df["inhigh3"] > 1.02]
print float(len(valid_df))/len(df)
invalid_df = df[df["inhigh3"] <= 1.02]
print float(len(invalid_df))/len(df)

0.755711677823
0.244288322177


In [16]:
valid_df = df[df["inlow3"] > 0.98]
print float(len(valid_df))/len(df)
invalid_df = df[df["inlow3"] <= 0.98]
print float(len(invalid_df))/len(df)

0.265254190408
0.734745809592
